# SensRay Ray Tracing & Sensitivity Kernels Demo

This notebook demonstrates:
- Ray tracing with ObsPy/TauP integration
- Computing per-cell ray path lengths
- Calculating sensitivity kernels
- Visualizing rays and kernels in cross-sections

In [1]:
import numpy as np
from sensray import PlanetModel, CoordinateConverter

## 1. Setup Model and Mesh

Create a model and mesh for ray tracing experiments.

In [2]:
# Load model and create mesh
model = PlanetModel.from_standard_model('prem')
mesh = model.create_mesh(
    mesh_type='tetrahedral',
    mesh_size_km=300.0,
    populate_properties=['vp', 'vs', 'rho']
)
print(f"Created mesh: {mesh.mesh.n_cells} cells")

Generated tetrahedral mesh: 197612 cells, 32849 points
Populated properties: ['vp', 'vs', 'rho']
Created mesh: 197612 cells
Populated properties: ['vp', 'vs', 'rho']
Created mesh: 197612 cells


## 2. Define Source-Receiver Geometry

Set up a realistic earthquake-station pair.

In [3]:
# Define source (earthquake) and receiver (seismic station) locations
source_lat, source_lon, source_depth = 0.0, 0.0, 150.0  # Equator, 150 km depth
receiver_lat, receiver_lon = 30.0, 45.0  # Surface station

# Compute great-circle plane normal for cross-sections
plane_normal = CoordinateConverter.compute_gc_plane_normal(
    source_lat, source_lon, receiver_lat, receiver_lon
)
print(f"Source: ({source_lat}°, {source_lon}°, {source_depth} km)")
print(f"Receiver: ({receiver_lat}°, {receiver_lon}°, 0 km)")
print(f"Great-circle plane normal: {plane_normal}")

Source: (0.0°, 0.0°, 150.0 km)
Receiver: (30.0°, 45.0°, 0 km)
Great-circle plane normal: (0.0, -0.6324555320336758, 0.7745966692414834)


## 3. Ray Tracing with TauP

Compute ray paths for different seismic phases.

In [4]:
# Get ray paths for P and S waves
rays = model.taupy_model.get_ray_paths_geo(
    source_depth_in_km=source_depth,
    source_latitude_in_deg=source_lat,
    source_longitude_in_deg=source_lon,
    receiver_latitude_in_deg=receiver_lat,
    receiver_longitude_in_deg=receiver_lon,
    phase_list=["P", "S", "ScS"]
)

print(f"Found {len(rays)} ray paths:")
for i, ray in enumerate(rays):
    print(f"  {i+1}. {ray.phase.name}: {ray.time:.2f} s, {len(ray.path)} points")

Building obspy.taup model for '/disks/data/PhD/masters/SensRay/sensray/models/prem.nd' ...
filename = /disks/data/PhD/masters/SensRay/sensray/models/prem.nd
Done reading velocity model.
Radius of model . is 6371.0
Using parameters provided in TauP_config.ini (or defaults if not) to call SlownessModel...
Parameters are:
taup.create.min_delta_p = 0.1 sec / radian
taup.create.max_delta_p = 11.0 sec / radian
taup.create.max_depth_interval = 115.0 kilometers
taup.create.max_range_interval = 0.04363323129985824 degrees
taup.create.max_interp_error = 0.05 seconds
taup.create.allow_inner_core_s = True
Slow model  553 P layers,646 S layers
Done calculating Tau branches.
Done Saving /tmp/prem.npz
Method run is done, but not necessarily successful.
Found 3 ray paths:
  1. P: 535.08 s, 227 points
  2. S: 969.32 s, 276 points
  3. ScS: 1110.00 s, 512 points
Parameters are:
taup.create.min_delta_p = 0.1 sec / radian
taup.create.max_delta_p = 11.0 sec / radian
taup.create.max_depth_interval = 115.0 k

## 4. Compute Ray Path Lengths

Calculate how much each ray travels through each mesh cell.

In [5]:
# Compute and store path lengths for each ray
P_ray = rays[0]  # First ray (P wave)
S_ray = rays[1] if len(rays) > 1 else rays[0]  # Second ray (S wave)

# Method 1: Simple computation and storage
P_lengths = mesh.add_ray_to_mesh(P_ray, "P_wave")
S_lengths = mesh.add_ray_to_mesh(S_ray, "S_wave")

print(f"P wave: {P_lengths.sum():.1f} km total, {np.count_nonzero(P_lengths)} cells")
print(f"S wave: {S_lengths.sum():.1f} km total, {np.count_nonzero(S_lengths)} cells")

# Show stored properties
ray_keys = [k for k in mesh.mesh.cell_data.keys() if 'ray_' in k]
print(f"Stored ray properties: {ray_keys}")

Stored ray path lengths as cell data: 'ray_P_wave_P_lengths'
Stored ray path lengths as cell data: 'ray_S_wave_S_lengths'
P wave: 5750.4 km total, 61 cells
S wave: 5728.9 km total, 59 cells
Stored ray properties: ['ray_P_wave_P_lengths', 'ray_S_wave_S_lengths']
Stored ray path lengths as cell data: 'ray_S_wave_S_lengths'
P wave: 5750.4 km total, 61 cells
S wave: 5728.9 km total, 59 cells
Stored ray properties: ['ray_P_wave_P_lengths', 'ray_S_wave_S_lengths']


## 5. Sensitivity Kernels

Compute travel-time sensitivity kernels: K = -L / v² for each cell.

In [6]:
# Compute sensitivity kernels for P and S waves
P_kernel = mesh.compute_sensitivity_kernel(
    P_ray, property_name='vp', attach_name='K_P_vp', epsilon=1e-6
)
S_kernel = mesh.compute_sensitivity_kernel(
    S_ray, property_name='vs', attach_name='K_S_vs', epsilon=1e-6
)

print(f"P kernel range: {P_kernel.min():.6f} to {P_kernel.max():.6f} s²/km³")
print(f"S kernel range: {S_kernel.min():.6f} to {S_kernel.max():.6f} s²/km³")
print(f"Non-zero P kernel cells: {np.count_nonzero(P_kernel)}")
print(f"Non-zero S kernel cells: {np.count_nonzero(S_kernel)}")

Stored sensitivity kernel as cell data: 'K_P_vp'
Stored sensitivity kernel as cell data: 'K_S_vs'
P kernel range: -3.217215 to -0.000000 s²/km³
S kernel range: -10.535492 to -0.000000 s²/km³
Non-zero P kernel cells: 61
Non-zero S kernel cells: 59
Stored sensitivity kernel as cell data: 'K_S_vs'
P kernel range: -3.217215 to -0.000000 s²/km³
S kernel range: -10.535492 to -0.000000 s²/km³
Non-zero P kernel cells: 61
Non-zero S kernel cells: 59


## 6. Multiple Ray Kernels

Combine kernels from multiple rays for enhanced sensitivity.

In [7]:
# Sum kernels from multiple rays
if len(rays) >= 2:
    combined_kernel = mesh.compute_sensitivity_kernels_for_rays(
        rays[:2],  # Use first two rays
        property_name='vp',
        attach_name='K_combined_vp',
        accumulate='sum'
    )
    print(f"Combined kernel range: {combined_kernel.min():.6f} to {combined_kernel.max():.6f}")
    print(f"Combined kernel non-zero cells: {np.count_nonzero(combined_kernel)}")

Stored summed sensitivity kernel as cell data: 'K_combined_vp'
Combined kernel range: -6.304745 to 0.000000
Combined kernel non-zero cells: 74


## 7. Visualization

Create cross-sections showing background velocity, ray paths, and sensitivity kernels.

In [8]:
# Cross-section showing background Vp
print("Background P-wave velocity:")
plotter1 = mesh.plot_cross_section(
    plane_normal=plane_normal,
    property_name='vp',
    show_rays=[P_ray]  # Overlay ray path
)
plotter1.camera.position = (8000, 6000, 10000)
plotter1.show()

Background P-wave velocity:


Widget(value='<iframe src="http://localhost:35859/index.html?ui=P_0x7f1aff081c10_0&reconnect=auto" class="pyvi…

In [9]:
# Cross-section showing P-wave ray path lengths
print("P-wave path lengths through cells:")
plotter2 = mesh.plot_cross_section(plane_normal=plane_normal,
                                   property_name='K_P_vp',
                                   show_rays=[P_ray])
plotter2.camera.position = (8000, 6000, 10000)
plotter2.show()

P-wave path lengths through cells:


Widget(value='<iframe src="http://localhost:35859/index.html?ui=P_0x7f1aff081890_1&reconnect=auto" class="pyvi…

## 8. Export Results

Save mesh with all computed properties for further analysis.

In [11]:
# Save mesh with rays and kernels
mesh.save('prem_tet_rays_kernels_demo')

# Show what was saved
info = mesh.list_properties(show_stats=False)
print(f"Saved {len(info['cell_data'])} properties to VTU file:")
for prop in info['cell_data'].keys():
    print(f"  - {prop}")

print("\nFiles created:")
print("  - prem_tet_rays_kernels_demo.vtu (mesh + all data)")
print("  - prem_tet_rays_kernels_demo_metadata.json (property list)")

Saved mesh to prem_tet_rays_kernels_demo.vtu
Saved metadata to prem_tet_rays_kernels_demo_metadata.json
Mesh properties summary:
  cell_data keys: ['vp', 'vs', 'rho', 'vtkOriginalCellIds', 'ray_P_wave_P_lengths', 'ray_S_wave_S_lengths', 'K_P_vp', 'K_S_vs', 'K_combined_vp']
Saved 9 properties to VTU file:
  - vp
  - vs
  - rho
  - vtkOriginalCellIds
  - ray_P_wave_P_lengths
  - ray_S_wave_S_lengths
  - K_P_vp
  - K_S_vs
  - K_combined_vp

Files created:
  - prem_tet_rays_kernels_demo.vtu (mesh + all data)
  - prem_tet_rays_kernels_demo_metadata.json (property list)


## Summary

This demo showed:
- Ray tracing with TauP for P, S, and ScS phases
- Computing per-cell ray path lengths
- Calculating sensitivity kernels K = -L/v²
- Combining multiple ray kernels
- Visualizing rays, path lengths, and kernels in cross-sections
- Exporting results to VTU format

**Key Methods:**
- `mesh.add_ray_to_mesh(ray, name)` - compute and store ray lengths
- `mesh.compute_sensitivity_kernel(ray, property, name)` - compute kernels
- `mesh.plot_cross_section_with_rays()` - visualize ray properties
- `mesh.save(path)` - export all data

Next: See `03_advanced_workflows.ipynb` for batch processing and analysis.